# Classification Approaches Comparison to Spam Detection
In this notebook different classification methods to **spam detection** are compared.
The algorithms compared are:
- Naive Bayes
- Support Vector Machine
- Random Forest
- K Nearest Neighbor
- Decision Tree

Each method is tested on:
- Raw email text
- Cleaned email text
- Stemmed email text
- Lemmatized email text

Also two different vectorizers methods are tested:
- Term Frequency - Inverse Document Frequency (TF-IDF) Vectorizer
- Count Vectorizer

The purpose of this notebook is to generate many graphs comparing the algorithms performance (using different input data and vectorizer) in order to find the best suited approaches for this task.

### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
from random import shuffle
%matplotlib inline
matplotlib.use('Agg')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Vectorizing dataset: 
In order to apply classification algorithms to our data, we need to transform the alphanumeric emails text into vectors which are able to represent the information contained in an email using a numeric format.
The vectorizers tested are:
- **Count vectorizer**: one of the simplest vectorization methods that uses is bag-of-words (BoW) representation. A BoW vector has the length of the entire vocabulary — that is, the set of unique words in the corpus. The vector’s values represent the frequency with which each word appears in a given text passage.

- **TF-IDF vectorizer**: this techinque attempts to give higher relevance scores to words that occur in fewer documents within the corpus. TF-IDF measures the frequency of a word in a text against its overall frequency in the corpus.


In [3]:
# Dataset is read from the csv
dataset = pd.read_csv("EmailDatasetCleaned.csv", index_col=0)

dataset.columns = ['email', 'label', 'length', 'clean_email_text', 
                   'tokenized_email_text', 'nonstop_email_text', 'stemmed_email_text', 
                   'lemmatized_email_text']

In [4]:
# Random samples from the dataset are printed
dataset = dataset.sample(frac=1)
dataset.head(5)

,email,label,length,clean_email_text,tokenized_email_text,nonstop_email_text,stemmed_email_text,lemmatized_email_text
24976,Subject: college degree online !\n,spam,33,subject college degree online,"['subject', 'college', 'degree', 'online']","['subject', 'college', 'degree', 'online']","['subject', 'colleg', 'degre', 'onlin']","['subject', 'college', 'degree', 'online']"
21166,Subject: purchase steroids online\n hi !\n the...,spam,591,subject purchase steroids online hi the stero...,"['subject', 'purchase', 'steroids', 'online', ...","['subject', 'purchase', 'steroids', 'online', ...","['subject', 'purchas', 'steroid', 'onlin', 'hi...","['subject', 'purchase', 'steroid', 'online', '..."
5353,Subject: moneycentral : 6 routes to retire ric...,ham,60,subject moneycentral routes to retire rich e...,"['subject', 'moneycentral', 'routes', 'to', 'r...","['subject', 'moneycentral', 'routes', 'retire'...","['subject', 'moneycentr', 'rout', 'retir', 'ri...","['subject', 'moneycentral', 'route', 'retire',..."
9827,Subject: enron / hpl actuals for february 2 - ...,ham,222,subject enron hpl actuals for february f...,"['subject', 'enron', 'hpl', 'actuals', 'for', ...","['subject', 'enron', 'hpl', 'actuals', 'februa...","['subject', 'enron', 'hpl', 'actual', 'februar...","['subject', 'enron', 'hpl', 'actuals', 'februa..."
12128,Subject: epe lending / day - ahead short for 8...,ham,1258,subject epe lending day ahead short for w...,"['subject', 'epe', 'lending', 'day', 'ahead', ...","['subject', 'epe', 'lending', 'day', 'ahead', ...","['subject', 'epe', 'lend', 'day', 'ahead', 'sh...","['subject', 'epe', 'lending', 'day', 'ahead', ..."


In [5]:
# Vectorizer costants
TFIDF = 0
COUNT = 1

# Algorithm constants
BAYES = 0
SVM = 1
RF = 2
KNN = 3
DT = 4

# Different formats of the input emails
inputs = {elem: i for i,elem in enumerate(['email','clean_email_text',
                                           'stemmed_email_text','lemmatized_email_text'])}

# Matrix containing numeric features using different formats and vectorizers
features = [[] for x in range(2)]

# 4 dimensional array containing scores for each algorithm, using different mail formats and vectorizers
score = [[[[] for z in range(4)] for y in range(2)] for x in range(5)]

In [6]:
tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

for type in inputs.keys():
    if type == 'email' or type == 'clean_email_text':
        emails = [o for o in dataset[type]]
    else:
        emails = [o.strip('[]\'').replace('\', \'',' ') for o in dataset[type]]
    features[TFIDF].append(tfidf_vectorizer.fit_transform(emails))
    features[COUNT].append(count_vectorizer.fit_transform(emails))

## Applying Classification Algorithms for Comparative Analysis
In this section the classification algorithms for comparative analysis are applied to see which one will yield the best results in terms of accuracy, precision and recall.

In [7]:
# Train and test matrices are initialized
X_train = [[] for x in range(2)]
X_test = [[] for x in range(2)]
y_train = [[] for x in range(2)]
y_test = [[] for x in range(2)]

In [8]:
# Train-Test Splits are performed
for i in range(2):
    for j in inputs.values() :
        a, b, c, d = (train_test_split(features[i][j], dataset['label'], test_size=0.2, random_state=111))
        X_train[i].append(a)
        X_test[i].append(b)
        y_train[i].append(c)
        y_test[i].append(d)

### Naive Bayes

It is a classification technique based on Bayes' Theorem with an assumption of independence among predictors. In simple terms, a Naive Bayes classifier assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature.

In [9]:
for i in range(2): # for each vectorizer
    for j in inputs.values() : # for each input format
        Xtrain = X_train[i][j]
        Xtest = X_test[i][j]
        ytrain = y_train[i][j]
        ytest = y_test[i][j]
        clf = MultinomialNB(alpha=0.2) # classifier is initialized
        clf.fit(Xtrain, ytrain) # training is performed
        y_pred_nb = clf.predict(Xtest) # testing
        # Metrics are saved
        acc = round(accuracy_score(ytest,y_pred_nb),5)
        pre, rec, _, _ = precision_recall_fscore_support(ytest, y_pred_nb, pos_label='spam', average='binary')
        score[BAYES][i][j] = (acc, round(pre,5), round(rec,5))

### Support Vector Machine

SVM or Support Vector Machine is a linear model for classification and regression problems. It can solve linear and non-linear problems and work well for many practical problems. The idea of SVM is simple: The algorithm creates a line or a hyperplane which separates the dataset into classes.

In [10]:
svc = SVC(kernel='sigmoid', gamma=1.0) # classifier is initialized
for i in range(2): # for each vectorizer
    for j in inputs.values() : # for each input format
        Xtrain = X_train[i][j]
        Xtest = X_test[i][j]
        ytrain = y_train[i][j]
        ytest = y_test[i][j]
        svc.fit(Xtrain, ytrain) # training is performed
        y_pred_svm = svc.predict(Xtest) # testing
        # Metrics are saved
        acc = round(accuracy_score(ytest,y_pred_svm),5)
        pre, rec, _, _ = precision_recall_fscore_support(ytest, y_pred_svm, pos_label='spam', average='binary')
        score[SVM][i][j] = (acc, round(pre,5), round(rec,5))
        # DEBUG LINE: print("Run "+str(i)+","+str(j)+" = " + str(score[SVM][i][j]))

### Random Forest

The random forest is a classification algorithm consisting of many decisions trees. It uses bagging and feature randomness when building each individual tree to try to create an uncorrelated forest of trees whose prediction by committee is more accurate than that of any individual tree.

In [ ]:
for i in range(2): # for each vectorizer
    for j in inputs.values() : # for each input format
        Xtrain = X_train[i][j]
        Xtest = X_test[i][j]
        ytrain = y_train[i][j]
        ytest = y_test[i][j]
        rf = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1) # classifier is initialized
        rf_model = rf.fit(Xtrain, ytrain) # training is performed
        y_pred_rf = rf_model.predict(Xtest) # testing
        # Metrics are saved
        acc = round(accuracy_score(ytest,y_pred_rf),5)
        pre, rec, _, _ = precision_recall_fscore_support(ytest, y_pred_rf, pos_label='spam', average='binary')
        score[RF][i][j] = (acc, round(pre,5), round(rec,5))

### K Nearest Neighbors

K nearest neighbors is a simple algorithm that stores all available cases and classifies new cases based on a similarity measure (e.g., distance functions).

In [ ]:
for i in range(2): # for each vectorizer
    for j in inputs.values() : # for each input format
        Xtrain = X_train[i][j]
        Xtest = X_test[i][j]
        ytrain = y_train[i][j]
        ytest = y_test[i][j]
        knn = KNeighborsClassifier(n_neighbors=5, weights='uniform',algorithm='auto', 
                                   p=1, metric='euclidean', n_jobs=-1) # classifier is initialized
        knn.fit(Xtrain, ytrain) # training is performed
        y_pred_knn = knn.predict(Xtest) # testing
        # Metrics are saved
        acc = round(accuracy_score(ytest,y_pred_knn),5)
        pre, rec, _, _ = precision_recall_fscore_support(ytest, y_pred_knn, 
                                                         pos_label='spam', average='binary')
        score[KNN][i][j] = (acc, round(pre,5), round(rec,5))

### Decision Tree
It is a simple and effective predictive modeling approach used in many classification and regression problems. The training process involves building a tree structure in which each branch recursively splits the dataset into two subsets based on the values of a selected feature. The leaves of the tree represent classifications indicating the predicted labels of the instances.


In [ ]:
for i in range(2): # for each vectorizer
    for j in inputs.values() : # for each input format
        Xtrain = X_train[i][j]
        Xtest = X_test[i][j]
        ytrain = y_train[i][j]
        ytest = y_test[i][j]
        dt = DecisionTreeClassifier() # classifier is initialized
        dt.fit(Xtrain, ytrain) # training is performed
        y_pred_dt = dt.predict(Xtest) # testing
        # Metrics are saved
        acc = round(accuracy_score(ytest,y_pred_dt),5)
        pre, rec, _, _ = precision_recall_fscore_support(ytest, y_pred_dt, 
                                                         pos_label='spam', average='binary')
        score[DT][i][j] = (acc, round(pre,5), round(rec,5))

## Results
In this section results are aggregated into graphs and saved as images in the Result directory.
In order to decide the best approaches, three metrics are considered:
- **Accuracy**: it is the fraction of successful predictions with respect to the total number of emails.
In our case the dataset is quite balanced so accuracy should be a reliable metric.

- **Precision**: it is the fraction of spam emails classified as spam with respect to the total number of emails classified as spam. The better the precision, the lower are false positive samples.

- **Recall**: it is the fraction of spam emails classified as spam with respect to the total number of spam emails. The better the recall, the lower are false negative samples.

In [ ]:
# This function allows to change the format of the score matrix to be used for plotting results
def change_format(matrix):
  a = np.concatenate((matrix[0],matrix[1]))
  b = np.transpose(a)
  b = [tuple(x) for x in b]
  return b

# Dictionary with the classifiers and their indexes
classifiers = {"Bayes":0,"SVM":1,"Random Forest":2,"KNN":3,"Decision Tree":4}

# Tuple with different combinations of vectorizer - email format.
formats = ("TFIDF - Raw", "TFIDF - Clean", "TFIDF - Stemmed","TFIDF - Lemmatized",
            "COUNT - Raw", "COUNT - Clean", "COUNT - Stemmed","COUNT - Lemmatized")

# Lists to save best approaches for accuracy/precisio/recall
best_acc = []
best_pre = []
best_rec = []
best_acc_method = []
best_pre_method = []
best_rec_method = []

In [ ]:
# This code plots accuracies, precision and recalls for a single algorithm considering all the 
# vecorizer-format combinations.

for elem in classifiers:

    output_data = change_format(np.copy(score[classifiers[elem]]))

    metrics = {
        'Accuracy': output_data[0],
        'Precision': output_data[1],
        'Recall': output_data[2],
    }

    best_acc.append(np.max(metrics["Accuracy"]))
    best_acc_method.append(formats[np.argmax(metrics["Accuracy"])])

    best_pre.append(np.max(metrics["Precision"]))
    best_pre_method.append(formats[np.argmax(metrics["Precision"])])

    best_rec.append(np.max(metrics["Recall"]))
    best_rec_method.append(formats[np.argmax(metrics["Recall"])])

    x = np.arange(len(formats))
    width = 0.25
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained',figsize=(20, 10))

    for attribute, measurement in metrics.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurement, width, label=attribute)
        ax.bar_label(rects, padding=3)
        multiplier += 1

    ax.set_title(elem + ' Scores')
    ax.set_xticks(x + width, formats)
    ax.legend(loc='lower left')
    ax.set_ylim(0.6, 1.03)

    plt.savefig('Results/'+elem+'.png')

In [ ]:
# This code plots, for each algorithm, the settings (vectorizer/format) that allows to achieve
# the best accuracy.

fig, ax = plt.subplots(layout='constrained',figsize=(14, 8))

algos = ['Bayes \n' + best_acc_method[0], 'SVM\n' + best_acc_method[1], 
         'Random Forest\n' + best_acc_method[2], 'KNN\n' + best_acc_method[3], 
         'Decision Tree\n' + best_acc_method[4]]

gr=sns.barplot(x=algos,y=best_acc,palette="GnBu")
ax.set_ylabel("Accuracy of ML Algorithms", fontsize=16)
ax.tick_params(labelsize=15)
gr.bar_label(gr.containers[0], fontsize=16)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)

plt.savefig('Results/ApproachesBestAccuracy.png')


In [ ]:
# This code plots, for each algorithm, the settings (vectorizer/format) that allows to achieve
# the best precision.

fig, ax = plt.subplots(layout='constrained',figsize=(14, 8))

algos = ['Bayes \n' + best_pre_method[0], 'SVM\n' + best_pre_method[1], 
         'Random Forest\n' + best_pre_method[2], 'KNN\n' + best_pre_method[3], 
         'Decision Tree\n' + best_pre_method[4]]

gr=sns.barplot(x=algos,y=best_pre,palette="GnBu")
ax.set_ylabel("Precision of ML Algorithms", fontsize=16)
ax.tick_params(labelsize=15)
gr.bar_label(gr.containers[0], fontsize=16)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)

plt.savefig('Results/ApproachesBestPrecision.png')

In [ ]:
# This code plots, for each algorithm, the settings (vectorizer/format) that allows to achieve
# the best recall.

fig, ax = plt.subplots(layout='constrained',figsize=(14, 8))

algos = ['Bayes \n' + best_rec_method[0], 'SVM\n' + best_rec_method[1], 
         'Random Forest\n' + best_rec_method[2], 'KNN\n' + best_rec_method[3], 
         'Decision Tree\n' + best_rec_method[4]]

gr=sns.barplot(x=algos,y=best_rec,palette="GnBu")
ax.set_ylabel("Recall of ML Algorithms", fontsize=16)
ax.tick_params(labelsize=15)
gr.bar_label(gr.containers[0], fontsize=16)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)

plt.savefig('Results/ApproachesBestRecall.png')